<a href="https://colab.research.google.com/github/Thisun1997/Classification-of-Anomalies-in-Gastrointestinal-Tract/blob/master/PCII_final.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!wget https://datasets.simula.no/kvasir/data/kvasir-dataset-v2.zip
!unzip -q "/content/kvasir-dataset-v2.zip" -d /content/


In [ ]:
import numpy as np
import keras
from keras import backend as K
from keras.models import Sequential
from keras.layers import Activation
from keras.layers import Dropout
from tqdm import tqdm
from keras.layers.core import Dense, Flatten
from keras.optimizers import Adam
from keras.metrics import categorical_crossentropy
from keras.preprocessing.image import ImageDataGenerator
from keras.layers.normalization import BatchNormalization
from keras.layers.convolutional import *
from matplotlib import pyplot as plt
from sklearn.metrics import confusion_matrix
import itertools
import matplotlib.pyplot as plt
from keras import models
from keras import layers
from keras import optimizers

In [ ]:
image_size = 224

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
from keras.applications import VGG19

# pre trained model
pre_model = VGG19(weights='imagenet', include_top=False, input_shape=(image_size, image_size, 3))

In [ ]:
# # only train last layer
for layer in pre_model.layers[:-5]:
    layer.trainable = False
    
for layer in pre_model.layers:
  print(layer, layer.trainable)

<keras.engine.input_layer.InputLayer object at 0x7f06e39746a0> False
<keras.layers.convolutional.Conv2D object at 0x7f06e3974630> False
<keras.layers.convolutional.Conv2D object at 0x7f06f178b9b0> False
<keras.layers.pooling.MaxPooling2D object at 0x7f06d4359160> False
<keras.layers.convolutional.Conv2D object at 0x7f06d4360898> False
<keras.layers.convolutional.Conv2D object at 0x7f06d4366978> False
<keras.layers.pooling.MaxPooling2D object at 0x7f06d436b6d8> False
<keras.layers.convolutional.Conv2D object at 0x7f06d437a710> False
<keras.layers.convolutional.Conv2D object at 0x7f06d437af28> False
<keras.layers.convolutional.Conv2D object at 0x7f06d4382e10> False
<keras.layers.convolutional.Conv2D object at 0x7f06d41045f8> False
<keras.layers.pooling.MaxPooling2D object at 0x7f06d410f5f8> False
<keras.layers.convolutional.Conv2D object at 0x7f06d4118630> False
<keras.layers.convolutional.Conv2D object at 0x7f06d411e7b8> False
<keras.layers.convolutional.Conv2D object at 0x7f06d4124518>

In [ ]:
model = models.Sequential()

model.add(pre_model)

model.add(Conv2D(1024, (3,3), activation='relu'))
model.add(MaxPooling2D(pool_size = (2,2)))
model.add(Flatten())
model.add(Dense(2048, activation='relu'))
model.add(Dropout(0.6))
model.add(Dense(8, activation='softmax')) 

model.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
vgg19 (Model)                (None, 7, 7, 512)         20024384  
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 5, 5, 1024)        4719616   
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 2, 2, 1024)        0         
_________________________________________________________________
flatten_2 (Flatten)          (None, 4096)              0         
_________________________________________________________________
dense_3 (Dense)              (None, 2048)              8390656   
_________________________________________________________________
dropout_2 (Dropout)          (None, 2048)              0         
_________________________________________________________________
dense_4 (Dense)              (None, 8)                

In [ ]:
# load data
data_dir = 'kvasir-dataset-v2'

from keras.preprocessing.image import ImageDataGenerator

data_gen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest',
    validation_split=0.125)

n_batch_train = 100
n_batch_val = 10

train_gen = data_gen.flow_from_directory(
    data_dir,
    target_size=(image_size, image_size),
    batch_size=n_batch_train,
    class_mode='categorical',
    seed=15,
    shuffle=True,
    subset='training')

val_gen = data_gen.flow_from_directory(
    data_dir,
    target_size=(image_size, image_size),
    batch_size=n_batch_val,
    class_mode='categorical',
    seed=15,
    shuffle=True,
    subset='validation')

Found 7000 images belonging to 8 classes.
Found 1000 images belonging to 8 classes.


In [ ]:
model.compile(loss='categorical_crossentropy',
             optimizer=optimizers.RMSprop(lr=2e-5),
             metrics=['acc'])

history = model.fit_generator(
    train_gen,
    steps_per_epoch=train_gen.samples/train_gen.batch_size,
    epochs=10,
    validation_data=val_gen,
    validation_steps=val_gen.samples/val_gen.batch_size,
    verbose=1)



Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


Epoch 1/10
70/70 [==============================] - 239s 3s/step - loss: 1.4184 - acc: 0.4310 - val_loss: 0.8958 - val_acc: 0.6600
Epoch 2/10
70/70 [==============================] - 201s 3s/step - loss: 0.8401 - acc: 0.6636 - val_loss: 0.7309 - val_acc: 0.7030
Epoch 3/10
70/70 [==============================] - 206s 3s/step - loss: 0.6802 - acc: 0.7306 - val_loss: 0.6344 - val_acc: 0.7370
Epoch 4/10
70/70 [==============================] - 199s 3s/step - loss: 0.5865 - acc: 0.7717 - val_loss: 0.4986 - val_acc: 0.8080
Epoch 5/10
70/70 [==============================] - 197s 3s/step - loss: 0.5178 - acc: 0.7907 - val_loss: 0.4943 - val_acc: 0.8060
Epoch 6/10
70/70 [==============================] - 196s 3s/step - loss: 0.4792 - acc: 0.8101 - val_loss: 0.5079 - val_acc: 0.8040
Epoch 7/10
70/70 [==============================] - 193s 3s/step - loss: 0.4505 - acc: 0.8231 - val_loss: 0.5149 - v